In [3]:
import os,random
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from keras.utils import np_utils
import keras.models as models
from keras.layers import Input
from keras.layers.core import Reshape,Dense,Dropout,Activation,Flatten
from keras.layers.noise import GaussianNoise
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.regularizers import *
from keras.optimizers import adam
from keras.optimizers import adagrad
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import seaborn as sns
import pickle as cPickle
import random, sys, keras
from keras import backend as K
if keras.backend.backend() == 'tensorflow':
    # Access TensorFlow-specific functionality directly
    from tensorflow.python.keras import backend as K
else:
    # Use default backend's functionality
    import keras.backend as K
import tensorflow as tf

In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  0


In [5]:
file_path = r"C:\Users\kshre\Downloads\DNN for MC\RML2016.10a_dict.pkl"
Xd = cPickle.load(open(file_path,'rb'), encoding='latin1')
snrs,mods = map(lambda j: sorted(list(set(map(lambda x: x[j], Xd.keys())))), [1,0])
X = []  
lbl = []
for mod in mods:
    for snr in snrs:
        X.append(Xd[(mod,snr)])
        for i in range(Xd[(mod,snr)].shape[0]):  lbl.append((mod,snr))
X = np.vstack(X)
print("shape of X", np.shape(X))

shape of X (220000, 2, 128)


In [6]:
print(len(X[0][0]))
#X[0][i] has two values for i, one representing real part and the other representing complex part of the signal
print(lbl[0])

128
('8PSK', -20)


In [7]:
np.random.seed(2016)
n_examples = X.shape[0]
n_train = n_examples // 2
train_idx = np.random.choice(range(0,n_examples), size=n_train, replace=False)
test_idx = list(set(range(0,n_examples))-set(train_idx))
X_train = X[train_idx]
X_test =  X[test_idx]
def to_onehot(yy):
    yy1 = np.zeros([len(yy), max(yy)+1])
    yy1[np.arange(len(yy)),yy] = 1
    return yy1
Y_train = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), train_idx)))
Y_test = to_onehot(list(map(lambda x: mods.index(lbl[x][0]), test_idx)))

In [8]:
in_shp = list(X_train.shape[1:])
print (X_train.shape, in_shp)
classes = mods
# Datapoint is a tuple of two arrays of size 128 each, representing the real and imaginary parts of the signal.
# The label associated with each data point is a tuple containing the modulation scheme and SNR value corresponding to that signal.

(110000, 2, 128) [2, 128]


In [9]:
#VT_CNN2

from keras.layers import Reshape, Conv2D, Activation, Flatten, Dense
from keras.models import Sequential
from keras.layers import ZeroPadding2D


# Build VT-CNN2 Neural Net model using Keras primitives -- 
#  - Reshape [N,2,128] to [N,1,2,128] on input
#  - Pass through 2 2DConv/ReLu layers
#  - Pass through 2 Dense layers (ReLu and Softmax) 
#  - Perform categorical cross entropy optimization
dr = 0.5
model = models.Sequential()
model.add(Reshape([1]+in_shp, input_shape=in_shp))
model.add(ZeroPadding2D((0,2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform", data_format="channels_first", name="conv1", activation="relu", padding="valid", filters=256, kernel_size=(1, 3)))
model.add(Dropout(dr))
model.add(ZeroPadding2D((0,2), data_format="channels_first"))
model.add(Conv2D(kernel_initializer="glorot_uniform",data_format="channels_first", name="conv2", activation="relu", padding="valid", filters=80, kernel_size=(2, 3)))
model.add(Dropout(dr))
model.add(Flatten())
model.add(Dense(256, kernel_initializer="he_normal", activation="relu", name="dense1"))
model.add(Dropout(dr))
model.add(Dense(len(classes), kernel_initializer="he_normal", name="dense2"))
model.add(Activation('softmax'))
model.add(Reshape([len(classes)]))
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape (Reshape)           (None, 1, 2, 128)         0         
                                                                 
 zero_padding2d (ZeroPadding  (None, 1, 2, 132)        0         
 2D)                                                             
                                                                 
 conv1 (Conv2D)              (None, 256, 2, 130)       1024      
                                                                 
 dropout (Dropout)           (None, 256, 2, 130)       0         
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 256, 2, 134)      0         
 ng2D)                                                           
                                                                 
 conv2 (Conv2D)              (None, 80, 1, 132)        1

In [10]:
count = 1
for i in model.layers:
    print(count," ",i.name," ",i.input_shape,"  ",i.output_shape)
    count+=1

1   reshape   (None, 2, 128)    (None, 1, 2, 128)
2   zero_padding2d   (None, 1, 2, 128)    (None, 1, 2, 132)
3   conv1   (None, 1, 2, 132)    (None, 256, 2, 130)
4   dropout   (None, 256, 2, 130)    (None, 256, 2, 130)
5   zero_padding2d_1   (None, 256, 2, 130)    (None, 256, 2, 134)
6   conv2   (None, 256, 2, 134)    (None, 80, 1, 132)
7   dropout_1   (None, 80, 1, 132)    (None, 80, 1, 132)
8   flatten   (None, 80, 1, 132)    (None, 10560)
9   dense1   (None, 10560)    (None, 256)
10   dropout_2   (None, 256)    (None, 256)
11   dense2   (None, 256)    (None, 11)
12   activation   (None, 11)    (None, 11)
13   reshape_1   (None, 11)    (None, 11)


In [11]:
nb_epoch = 100     # number of epochs to train on
batch_size = 1024  # training batch size

In [12]:
# Train the Model
# perform training ...
#   - call the main training loop in keras for our network+dataset

    # with arg_scope([layers.conv2d], activation_fn=self.activation_fn, data_format="NCHW"), \
        # arg_scope([layers.fully_connected], activation_fn=self.activation_fn):

filepath = 'convmodrecnets_CNN2_0.5.wts.h5'

history = model.fit(X_train,
    Y_train,
    batch_size=batch_size,
    epochs=nb_epoch,
    verbose=2,
    validation_data = (X_test, Y_test),
    callbacks = [
        keras.callbacks.ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto'),
        keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, verbose=0, mode='auto')
    ])
# we re-load the best weights once training is finished
model.load_weights(filepath)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'gradient_tape/sequential/conv2/Conv2D/Conv2DBackpropInput' defined at (most recent call last):
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\site-packages\traitlets\config\application.py", line 976, in launch_instance
      app.start()
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "c:\Users\kshre\AppData\Local\Programs\Python\Python310\lib\

In [ ]:
# Show simple version of performance
score = model.evaluate(X_test, Y_test, show_accuracy=True, verbose=0, batch_size=batch_size)
print(score)

In [ ]:
# Show loss curves 
plt.figure()
plt.title('Training performance')
plt.plot(history.epoch, history.history['loss'], label='train loss+error')
plt.plot(history.epoch, history.history['val_loss'], label='val_error')
plt.legend()

In [ ]:
def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues, labels=[]):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(labels))
    plt.xticks(tick_marks, labels, rotation=45)
    plt.yticks(tick_marks, labels)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:

# Plot confusion matrix
test_Y_hat = model.predict(X_test, batch_size=batch_size)
conf = np.zeros([len(classes),len(classes)])
confnorm = np.zeros([len(classes),len(classes)])
for i in range(0,X_test.shape[0]):
    j = list(Y_test[i,:]).index(1)
    k = int(np.argmax(test_Y_hat[i,:]))
    conf[j,k] = conf[j,k] + 1
for i in range(0,len(classes)):
    confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
plot_confusion_matrix(confnorm, labels=classes)

In [ ]:
# Plot confusion matrix
acc = {}
for snr in snrs:

    # extract classes @ SNR
    test_SNRs = map(lambda x: lbl[x][1], test_idx)
    test_X_i = X_test[np.where(np.array(test_SNRs)==snr)]
    test_Y_i = Y_test[np.where(np.array(test_SNRs)==snr)]    

    # estimate classes
    test_Y_i_hat = model.predict(test_X_i)
    conf = np.zeros([len(classes),len(classes)])
    confnorm = np.zeros([len(classes),len(classes)])
    for i in range(0,test_X_i.shape[0]):
        j = list(test_Y_i[i,:]).index(1)
        k = int(np.argmax(test_Y_i_hat[i,:]))
        conf[j,k] = conf[j,k] + 1
    for i in range(0,len(classes)):
        confnorm[i,:] = conf[i,:] / np.sum(conf[i,:])
    plt.figure()
    plot_confusion_matrix(confnorm, labels=classes, title="ConvNet Confusion Matrix (SNR=%d)"%(snr))
    
    cor = np.sum(np.diag(conf))
    ncor = np.sum(conf) - cor
    print("Overall Accuracy: ", cor / (cor+ncor))
    acc[snr] = 1.0*cor/(cor+ncor)

In [ ]:
# Save results to a pickle file for plotting later
print(acc)
fd = open('results_cnn2_d0.5.dat','wb')
cPickle.dump( ("CNN2", 0.5, acc) , fd )

In [ ]:
# Plot accuracy curve
plt.plot(snrs, map(lambda x: acc[x], snrs))
plt.xlabel("Signal to Noise Ratio")
plt.ylabel("Classification Accuracy")
plt.title("CNN2 Classification Accuracy on RadioML 2016.10 Alpha")